In [1]:
from inference.data_analysis import *

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
target_prop = "dft_e_hull"
test_set_types = ["test_set", "holdout_set_B_sites", "holdout_set_series"]

experimental_settings = [
    {"model_type": "CGCNN", "interpolation": False, "relaxed": False, "exp_id": 596732},
    {"model_type": "CGCNN", "interpolation": False, "relaxed": True, "exp_id": 596731},
    {"model_type": "CGCNN", "interpolation": True, "relaxed": False, "exp_id": 596670},
    {"model_type": "CGCNN", "interpolation": True, "relaxed": True, "exp_id": 596671},
    {"model_type": "Painn", "interpolation": True, "relaxed": False, "exp_id": 596674},
    {"model_type": "Painn", "interpolation": True, "relaxed": True, "exp_id": 596675},
    {"model_type": "e3nn", "interpolation": True, "relaxed": False, "exp_id": 596676},
    {"model_type": "e3nn", "interpolation": True, "relaxed": True, "exp_id": 596677},
    {"model_type": "e3nn_contrastive", "interpolation": True, "relaxed": False, "exp_id": 596672},
    {"model_type": "e3nn_contrastive", "interpolation": True, "relaxed": False, "exp_id": 597227},
    {"model_type": "e3nn_contrastive", "interpolation": True, "relaxed": True, "exp_id": 596673},
    {"model_type": "e3nn_contrastive", "interpolation": True, "relaxed": True, "exp_id": 597288},
]

series = [
    [["La", "Pr"], ["Y", "Ni"]],
    [["K", "Ba"], ["Ti", "Al"]],
    [["Y", "La"], ["In", "Mg"]],
    [["Mg", "Pr"], ["Ni", "V"]],
]

In [3]:
# plot_hex_all(target_prop, test_set_types, experimental_settings)

In [4]:
# plot_violin_all(target_prop, experimental_settings, series)

In [5]:
# plot_training_e3nn_contrastive(target_prop, experimental_settings)

In [6]:
num_best_models = 3
test_set_type = "holdout_set_series"

for experimental_setting in experimental_settings:
    sigopt_name = build_sigopt_name(target_prop, experimental_setting["relaxed"], experimental_setting["interpolation"], experimental_setting["model_type"])
    directory = "./best_models/" + experimental_setting["model_type"] + "/" + sigopt_name + "/" +str(experimental_setting["exp_id"])
    test_set_dfs = []
    
    for i in range(num_best_models):
        with open(directory + "/" + "best_" + str(i) + "/" + test_set_type + "_predictions.json") as f:
            test_set_dfs.append(pd.read_json(f).sort_values(by=["formula", "unrelaxed_cryst_id"]))

    compositions = list(set(test_set_dfs[0]["formula"]))

    # all_predicted_layered_rocksalt_diff = []
    # all_true_layered_rocksalt_diff = []
    all_predicted_energies_vs_groundstate = []
    all_true_energies_vs_groundstate = []

    for i in range(num_best_models):
        # predicted_layered_rocksalt_diff = []
        # true_layered_rocksalt_diff = []
        predicted_energies_vs_groundstate = []
        true_energies_vs_groundstate = []

        for composition in compositions:
            selected_df = test_set_dfs[i].loc[test_set_dfs[i]["formula"] == composition]
            
            adjusted_predicted_energies = list(np.array(selected_df["predicted_" + target_prop]) - selected_df["predicted_" + target_prop].min())
            predicted_energies_vs_groundstate = predicted_energies_vs_groundstate + adjusted_predicted_energies
            adjusted_true_energies = list(np.array(selected_df[target_prop]) - selected_df[target_prop].min())
            true_energies_vs_groundstate = true_energies_vs_groundstate + adjusted_true_energies
            
            ids = list(selected_df["unrelaxed_cryst_id"])
            # predicted_layered = selected_df.loc[selected_df["unrelaxed_cryst_id"] == min(ids)]["predicted_" + target_prop]
            # predicted_rocksalt = selected_df.loc[selected_df["unrelaxed_cryst_id"] == max(ids)]["predicted_" + target_prop]
            # true_layered = selected_df.loc[selected_df["unrelaxed_cryst_id"] == min(ids)][target_prop]
            # true_rocksalt = selected_df.loc[selected_df["unrelaxed_cryst_id"] == max(ids)][target_prop]
            # predicted_layered_rocksalt_diff.append(float(predicted_layered) - float(predicted_rocksalt))
            # true_layered_rocksalt_diff.append(float(true_layered) - float(true_rocksalt))

        # all_predicted_layered_rocksalt_diff.append(predicted_layered_rocksalt_diff)
        # all_true_layered_rocksalt_diff.append(true_layered_rocksalt_diff)
        all_predicted_energies_vs_groundstate.append(predicted_energies_vs_groundstate)
        all_true_energies_vs_groundstate.append(true_energies_vs_groundstate)

    # all_predicted_layered_rocksalt_diff = np.array(all_predicted_layered_rocksalt_diff).T
    # all_true_layered_rocksalt_diff = np.array(all_true_layered_rocksalt_diff).T
    all_predicted_energies_vs_groundstate = np.array(all_predicted_energies_vs_groundstate).T
    all_true_energies_vs_groundstate = np.array(all_true_energies_vs_groundstate).T

    # all_predicted_layered_rocksalt_diff_mean = np.mean(all_predicted_layered_rocksalt_diff, axis=1)
    # all_true_layered_rocksalt_diff_mean = np.mean(all_true_layered_rocksalt_diff, axis=1)

    # maes = []
    # for i in range(num_best_models):
    #     maes.append(mean_absolute_error(all_true_layered_rocksalt_diff[:,i], all_predicted_layered_rocksalt_diff[:,i]))
    # mae_mean = np.mean(maes)
    # mae_std = np.std(maes)

    # plot_hex(
    #     all_true_layered_rocksalt_diff_mean, all_predicted_layered_rocksalt_diff_mean, test_set_type, 
    #     experimental_setting, sigopt_name, str(experimental_setting["exp_id"]), 
    #     additional_string="layered_rocksalt",mae_mean=mae_mean, mae_std=mae_std
    #     )
    
    all_predicted_energies_vs_groundstate_mean = np.mean(all_predicted_energies_vs_groundstate, axis=1)
    all_true_energies_vs_groundstate_mean = np.mean(all_true_energies_vs_groundstate, axis=1)

    maes = []
    for i in range(num_best_models):
        maes.append(mean_absolute_error(all_true_energies_vs_groundstate[:,i], all_predicted_energies_vs_groundstate[:,i]))
    mae_mean = np.mean(maes)
    mae_std = np.std(maes)

    plot_hex(
        all_true_energies_vs_groundstate_mean, all_predicted_energies_vs_groundstate_mean, test_set_type,
        experimental_setting, sigopt_name, str(experimental_setting["exp_id"]),
        additional_string="energies_vs_groundstate", mae_mean=mae_mean, mae_std=mae_std
        )

Completed dft_e_hull_unrelaxed_CGCNN 596732
Completed dft_e_hull_relaxed_CGCNN 596731
Completed dft_e_hull_unrelaxed_interpolation_CGCNN 596670
Completed dft_e_hull_relaxed_interpolation_CGCNN 596671
Completed dft_e_hull_unrelaxed_interpolation_Painn 596674
Completed dft_e_hull_relaxed_interpolation_Painn 596675
Completed dft_e_hull_unrelaxed_interpolation_e3nn 596676
Completed dft_e_hull_relaxed_interpolation_e3nn 596677
Completed dft_e_hull_unrelaxed_interpolation_e3nn_contrastive 596672
Completed dft_e_hull_unrelaxed_interpolation_e3nn_contrastive 597227
Completed dft_e_hull_relaxed_interpolation_e3nn_contrastive 596673
Completed dft_e_hull_relaxed_interpolation_e3nn_contrastive 597288
